In [1]:
!pip install -q duckdb --upgrade


[notice] A new release of pip is available: 25.0.1 -> 25.2
[notice] To update, run: pip install --upgrade pip


In [2]:
# This CATALOG_URL works for the "docker compose" testing and development environment
# Change 'lakekeeper' if you are not running on "docker compose" (f. ex. 'localhost' if Lakekeeper is running locally).
CATALOG_URL = "http://lakekeeper:8181/catalog"
WAREHOUSE = "sepahram"

In [3]:
import duckdb

# Prepare Catalog

In [4]:
duckdb.sql("INSTALL ICEBERG;");
duckdb.sql("INSTALL httpfs;");
duckdb.sql("UPDATE EXTENSIONS;");
duckdb.sql("LOAD ICEBERG;");
duckdb.sql("LOAD httpfs;");

FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

In [5]:
duckdb.sql(f"""attach '{WAREHOUSE}' as my_datalake (
	type ICEBERG,
	ENDPOINT '{CATALOG_URL}',
	TOKEN ''
)
""");

## Read Tables
Make sure to run the `Pyiceberg.ipynb` Notebook first!

In [6]:
duckdb.sql("SELECT * FROM my_datalake.banking.source_transactions").show()

┌──────────────────────────────────────┬────────────────┬───────────────────────────────┬────────┬──────────┬───────────────────┬───────────────────────────────────┬─────────────────────────────┬─────────────────┬─────────────────┬─────────────┬──────────────────────────────────────┐
│            transactionId             │     userId     │           timestamp           │ amount │ currency │       city        │              country              │        merchantName         │  paymentMethod  │    ipAddress    │ voucherCode │             affiliateId              │
│               varchar                │    varchar     │   timestamp with time zone    │ double │ varchar  │      varchar      │              varchar              │           varchar           │     varchar     │     varchar     │   varchar   │               varchar                │
├──────────────────────────────────────┼────────────────┼───────────────────────────────┼────────┼──────────┼───────────────────┼────────────────

### 1. Total transaction volume per currency

This query helps you see how much money is flowing in each currency. It counts the number of transactions, the total sum of amounts, and the average transaction size for each currency.

In [8]:
duckdb.sql("""
    SELECT 
        currency, 
        COUNT(*) AS total_transactions, 
        SUM(amount) AS total_amount, 
        AVG(amount) AS avg_amount
    FROM my_datalake.banking.source_transactions
    GROUP BY currency
    ORDER BY total_amount DESC;
""")


┌──────────┬────────────────────┬──────────────┬───────────────────┐
│ currency │ total_transactions │ total_amount │    avg_amount     │
│ varchar  │       int64        │    double    │      double       │
├──────────┼────────────────────┼──────────────┼───────────────────┤
│ GBP      │                371 │    192231.63 │ 518.1445552560647 │
│ USD      │                379 │    187141.52 │ 493.7770976253298 │
└──────────┴────────────────────┴──────────────┴───────────────────┘

### 2. Top 10 merchants by total transaction amount

This query identifies which merchants generated the most revenue. It sums up the transaction amounts per merchant and also counts how many transactions each merchant had.

In [10]:
duckdb.sql("""
SELECT 
    merchantName, 
    SUM(amount) AS total_sales, 
    COUNT(*) AS transaction_count
FROM my_datalake.banking.source_transactions
GROUP BY merchantName
ORDER BY total_sales DESC
LIMIT 10;

""")


┌───────────────┬─────────────┬───────────────────┐
│ merchantName  │ total_sales │ transaction_count │
│    varchar    │   double    │       int64       │
├───────────────┼─────────────┼───────────────────┤
│ Nguyen PLC    │     2508.52 │                 3 │
│ Hill PLC      │     1754.41 │                 3 │
│ Reyes LLC     │     1564.77 │                 2 │
│ Jenkins PLC   │     1552.13 │                 2 │
│ Williams Inc  │     1289.28 │                 2 │
│ Henderson Inc │     1276.77 │                 2 │
│ Martinez PLC  │     1233.47 │                 2 │
│ Johnson Group │     1187.78 │                 2 │
│ Wilson Group  │      1011.5 │                 2 │
│ Harris Group  │     1008.46 │                 2 │
├───────────────┴─────────────┴───────────────────┤
│ 10 rows                               3 columns │
└─────────────────────────────────────────────────┘

### 3. Transactions distribution by payment method

This query shows how transactions are split between different payment methods (credit card, debit card, online transfer). It also calculates the percentage share of each method in the total dataset.

In [11]:
duckdb.sql("""
SELECT 
    paymentMethod, 
    COUNT(*) AS total_transactions, 
    ROUND(100.0 * COUNT(*) / SUM(COUNT(*)) OVER (), 2) AS percentage
FROM my_datalake.banking.source_transactions
GROUP BY paymentMethod
ORDER BY total_transactions DESC;
""")

┌─────────────────┬────────────────────┬────────────┐
│  paymentMethod  │ total_transactions │ percentage │
│     varchar     │       int64        │   double   │
├─────────────────┼────────────────────┼────────────┤
│ online_transfer │                351 │       35.1 │
│ credit_card     │                327 │       32.7 │
│ debit_card      │                322 │       32.2 │
└─────────────────┴────────────────────┴────────────┘

### 4. Daily transaction trend

This query aggregates transactions by day. It shows the daily count of transactions and the total transaction amount. Useful for time-series analysis and spotting peaks or drops in activity.

In [12]:
duckdb.sql("""
SELECT 
    DATE_TRUNC('day', timestamp) AS day, 
    COUNT(*) AS transactions, 
    SUM(amount) AS total_amount
FROM my_datalake.banking.source_transactions
GROUP BY day
ORDER BY day;

""")

┌──────────────────────────┬──────────────┬──────────────┐
│           day            │ transactions │ total_amount │
│ timestamp with time zone │    int64     │    double    │
├──────────────────────────┼──────────────┼──────────────┤
│ 2025-09-10 00:00:00+00   │         1000 │    514084.46 │
└──────────────────────────┴──────────────┴──────────────┘

### 5. Fraud-style detection: unusually high transactions

This query filters transactions above a certain threshold (here, amount > 900). It lists suspiciously high-value transactions for further investigation.

In [13]:
duckdb.sql("""
SELECT 
    transactionId, 
    userId, 
    amount, 
    currency, 
    merchantName, 
    paymentMethod, 
    timestamp
FROM my_datalake.banking.source_transactions
WHERE amount > 900
ORDER BY amount DESC;
    
""")

┌──────────────────────────────────────┬────────────────┬────────┬──────────┬───────────────────────────────┬─────────────────┬───────────────────────────────┐
│            transactionId             │     userId     │ amount │ currency │         merchantName          │  paymentMethod  │           timestamp           │
│               varchar                │    varchar     │ double │ varchar  │            varchar            │     varchar     │   timestamp with time zone    │
├──────────────────────────────────────┼────────────────┼────────┼──────────┼───────────────────────────────┼─────────────────┼───────────────────────────────┤
│ cf2f5d05-3c8b-4a4e-99b5-aa816a460e31 │ thomaslopez    │ 999.98 │ USD      │ Anderson LLC                  │ credit_card     │ 2025-09-10 14:15:03.42635+00  │
│ 97506059-9cb8-4901-9098-62ab5b2a7b65 │ christopher94  │ 999.48 │ USD      │ Smith, Carlson and Sandoval   │ credit_card     │ 2025-09-10 14:15:23.167811+00 │
│ d931f890-bba7-46f9-b16a-2dfdcfad75aa │

### 6.Rank users by total spending

This query finds the top spenders per country by ranking users based on how much they’ve spent.

In [14]:
duckdb.sql("""
SELECT 
    userId,
    country,
    SUM(amount) AS total_spent,
    RANK() OVER (PARTITION BY country ORDER BY SUM(amount) DESC) AS spending_rank
FROM my_datalake.banking.source_transactions
GROUP BY userId, country
ORDER BY country, spending_rank;
""")


┌────────────────┬─────────────┬─────────────┬───────────────┐
│     userId     │   country   │ total_spent │ spending_rank │
│    varchar     │   varchar   │   double    │     int64     │
├────────────────┼─────────────┼─────────────┼───────────────┤
│ aliciahoward   │ Afghanistan │      994.75 │             1 │
│ gschultz       │ Afghanistan │      848.11 │             2 │
│ clarksusan     │ Afghanistan │      744.16 │             3 │
│ nblanchard     │ Albania     │      912.56 │             1 │
│ ccollins       │ Albania     │      566.67 │             2 │
│ apage          │ Albania     │      560.87 │             3 │
│ johnsonjoshua  │ Albania     │      547.79 │             4 │
│ sara69         │ Algeria     │       850.1 │             1 │
│ raymond42      │ Algeria     │      729.84 │             2 │
│ kleinjennifer  │ Algeria     │      581.17 │             3 │
│      ·         │   ·         │         ·   │             · │
│      ·         │   ·         │         ·   │         

### 7.Running total of transactions per day

This query shows how transaction amounts accumulate over time (a running total trend).

In [19]:
duckdb.sql("""
SELECT 
    timestamp,
    amount,
    SUM(amount) OVER (ORDER BY timestamp ROWS BETWEEN UNBOUNDED PRECEDING AND CURRENT ROW) AS running_total
FROM my_datalake.banking.source_transactions
ORDER BY 1;
""")


┌───────────────────────────────┬────────┬────────────────────┐
│           timestamp           │ amount │   running_total    │
│   timestamp with time zone    │ double │       double       │
├───────────────────────────────┼────────┼────────────────────┤
│ 2025-09-10 14:07:16.187906+00 │  39.92 │              39.92 │
│ 2025-09-10 14:07:16.427611+00 │ 286.63 │             326.55 │
│ 2025-09-10 14:07:17.344155+00 │ 500.02 │  826.5699999999999 │
│ 2025-09-10 14:07:18.016434+00 │  62.87 │  889.4399999999999 │
│ 2025-09-10 14:07:18.185942+00 │ 828.68 │            1718.12 │
│ 2025-09-10 14:07:19.14759+00  │ 754.13 │            2472.25 │
│ 2025-09-10 14:07:19.826012+00 │ 332.61 │            2804.86 │
│ 2025-09-10 14:07:20.496162+00 │ 704.13 │ 3508.9900000000002 │
│ 2025-09-10 14:07:21.176865+00 │  565.0 │ 4073.9900000000002 │
│ 2025-09-10 14:07:21.70943+00  │  92.06 │            4166.05 │
│              ·                │    ·   │               ·    │
│              ·                │    ·  